In [1]:
print("test")
from transformers import AutoModelForCausalLM, AutoTokenizer


test


!pip install bert-score
!pip install sacrebleu
!pip install rouge

In [ ]:
finetuned_model = "./finetuned/policy-llama2-7b"

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained(finetuned_model)
tokenizer = AutoTokenizer.from_pretrained(finetuned_model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
rag_model = "./models_hf_with_rag/7B"

# Load the base  model and tokenizer
model_rag = AutoModelForCausalLM.from_pretrained(rag_model)
tokenizer_rag = AutoTokenizer.from_pretrained(rag_model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
base_model = "./models_hf/7B"

# Load the base  model and tokenizer
model_base = AutoModelForCausalLM.from_pretrained(base_model)
tokenizer_base = AutoTokenizer.from_pretrained(base_model)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# Generate text based on a given prompt
def generate_text(prompt, model, tokenizer, max_length=100):
    # Tokenize the prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)

    # Generate text based on the prompt
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, temperature=0.7)

    # Decode the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

# Example usage
# prompt = "What is the regulation around training powerful AI models in Europe?"
# generated_text = generate_text(prompt, model, tokenizer)
# print("Generated Text:", generated_text)

In [4]:
from bert_score import score
from sacrebleu.metrics import BLEU
from rouge import Rouge

class SimilarityEvaluator:
    def bert_score_calc(self, cands, ref1):
        P1, R1, F1 = score(cands, ref1, lang="en", verbose=True)
        return F1

    def bleu_score_calc(self, cands, ref1):
        bleu_scorer = BLEU()
        score1 = bleu_scorer.sentence_score(
            hypothesis=cands[0],
            references=ref1,
        )
        return score1.score/100

    def rouge_score_calc(self, cands, ref1):
        rouge_scorer = Rouge()
        score1 = rouge_scorer.get_scores(
            hyps=cands[0],
            refs=ref1[0],
        )
        # return score1[0]['rouge-1']['f']
        return score1[0]['rouge-l']['f']

    def evaluate_similarity(self, cands, ref1):
        bert_score = self.bert_score_calc(cands, ref1)
        bleu_score = self.bleu_score_calc(cands, ref1)
        rouge_score = self.rouge_score_calc(cands, ref1)
        return (bert_score, bleu_score, rouge_score)

In [5]:
import json
import csv

evaluator = SimilarityEvaluator() 

with open("merged_file_test.json", "r") as f, open("evaluation_scores_finetune.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Input", "Output", "Generated Answer", "BERT Score", "BLEU Score", "ROUGE Score"])
    
    datas = json.load(f)
    for data in datas:
        # Print or process the extracted input and output
        print("Input:", data["input"])
        print("Output:", data["output"])
        
        answer = generate_text(data["input"], model, tokenizer)
        answer = answer.replace(data["input"], "")
        print("Gen answer:", answer)
        eval_scores = evaluator.evaluate_similarity([data["output"]], [answer])
        print(eval_scores)
        writer.writerow([data["input"], data["output"], answer, eval_scores[0].tolist(), eval_scores[1], eval_scores[2]])
        

JSONDecodeError: Expecting value: line 2 column 1 (char 2)

In [ ]:
import json
import csv

evaluator = SimilarityEvaluator() 

with open("prompt_outputs2.csv", "r", encoding="utf-8") as f:
    datas = json.load(f)
    for data in datas:
        answer = generate_text(data["input"], model_base, tokenizer_base)
        answer = answer.replace(data["input"], "")
        eval_scores = evaluator.evaluate_similarity([data["output"]], [answer])
        print(eval_scores)
        
        # Open the CSV file in append mode
        open("evaluation_scores_base.csv", "w", newline="", encoding="utf-8") as csvfile:
            writer = csv.writer(csvfile)
            
            # Write to the CSV fiNle
            writer.writerow([data["input"], data["output"], answer, eval_scores[0].tolist(), eval_scores[1], eval_scores[2]])

In [ ]:
import json
import csv

evaluator = SimilarityEvaluator() 

with open("merged_file_test.json", "r") as f:
    datas = json.load(f)
    for data in datas:
        # Print or process the extracted input and output
        print("Input:", data["input"])
        print("Output:", data["output"])
        
        answer = generate_text(data["input"], model_rag, tokenizer_rag)
        answer = answer.replace(data["input"], "")
        answer = answer.split("Context:")[0].strip()
        print("Gen answer:", answer)
        eval_scores = evaluator.evaluate_similarity([data["output"]], [answer])
        print(eval_scores)
        
        # Open the CSV file in append mode
        with open("evaluation_scores_rag.csv", "a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            
            # Write to the CSV fiNle
            writer.writerow([data["input"], data["output"], answer, eval_scores[0].tolist(), eval_scores[1], eval_scores[2]])

/jet/home/achoudh2/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: What is the trajectory of Chinese AI governance, and what milestone is it approaching?
Output: Chinese AI governance is heading towards drafting a comprehensive national AI law, mirroring the evolution of internet governance regulations, with a potential draft release by late 2023 or 2024 and subsequent revisions involving key stakeholders.
Gen answer: The Chinese government has been actively promoting the development of AI, and has also been actively promoting the development of AI governance.
In 2017, the Chinese government issued the “National New Generation Artificial Intelligence Development Plan (2017-2020)”, which proposed to promote the development of AI governance


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

It is recommended to enable `effective_order` for sentence-level BLEU.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


done in 0.07 seconds, 14.82 sentences/sec
(tensor([0.8850]), 0.25540636397110394, 0.47999999539200017)
Input: What are some of the key principles outlined in the White House's Blueprint for an AI Bill of Rights? 
Output: The Blueprint emphasizes principles such as the right to safe automated systems, protection from algorithmic discrimination, data privacy, notice of AI use, and meaningful human alternatives 


In [ ]:
generated_text = generate_text(prompt, model, tokenizer)